# Part 2: Input Data - Graph Extraction

[![Notebook](https://shields.io/badge/notebook-access-green?logo=jupyter&style=for-the-badge)](https://github.com/cognitedata/neat/blob/main/docs/tutorial/notebooks/part-2-knowledge-graph-generation.ipynb)


* author: Nikola Vasiljevic, Anders Albert
* date: 2023-10-22

**Prerequisite**: 
 * Installed Python with `excel` dependency `pip install cognite-neat[excel]`
 * Completed tutorial [Part 1](part-1-data-model-generation.html)

**Content** This notebook represent Part 2 of NEAT Onboarding tutorial, in it we will demonstrate how to extract data into `neat`'s knowledge graph.

After importing or creating the *Rules* you are ready to extract some data.

`neat` supports multiple ways of ways of extracting data from a source into `neat`'s knowledge graph (aka `NeatGraphStore`), which is `neat`'s internal storage for data. Once the data has been extracted into `neat`, we can export it from `neat` to various formats such as `instances` and `asset` + `relationships`. 

With completion of this notebook you will be familiar with both Data Modeling Flow and Graph ETL Flow, as shown in the figure below:

Data modeling flow in `neat` are visually presented in the figure below:

![NEAT Flows](../../figs/neat-two_flows.png)

The two extraction methods we will cover in this tutorial are:
1. Generating mock data using `neat` mock module.
2. Manual creation with the help of the Graph Capturing Sheet.

These might seems like trivial use cases, however, both of these approaches has useful applications. First, if is often is useful to test how knowledge graph based on certain data model will function, for example, how queries would perform if we have very large knowledge graphs or very deep knowledge graph (many hops). In this case, creating a lot of mock data is useful. Another case is when we do not have knowledge graphs per se. Instead we have scattered and unconnected information which needs to be bring together to form knowledge graph. 




We will start with importing the same examples rules as we used in Part 1

In [1]:
from cognite.neat.rules import importer
from cognite.neat.rules.examples import power_grid_model

In [2]:
power_rules = importer.ExcelImporter(power_grid_model).to_rules()

## Extracting Mock Data

We start by inspecting the classes we have in the *Transformation Rules*

In [3]:
power_rules.classes

,description,cdf_resource_type,deprecated,class_id,class_name,parent_asset
0,None,Asset,False,GeographicalRegion,GeographicalRegion,None
1,A subset of a geographical region of a power s...,Asset,False,SubGeographicalRegion,SubGeographicalRegion,GeographicalRegion
2,A substation is a part of an electrical genera...,Asset,False,Substation,Substation,SubGeographicalRegion
3,None,Asset,False,Terminal,Terminal,Substation


We see that we have 4 different types of classes. To generate mock data for these classes, we write up the following dictionary.

In [4]:
class_count = {"GeographicalRegion":5,
               "SubGeographicalRegion":10,
               "Substation": 20,
               "Terminal": 60}

Then, we need to setup the `neat` knowledge graph

In [5]:
from cognite.neat.graph import NeatGraphStore

store = NeatGraphStore.from_rules(power_rules)

All data in `neat` is represented as triples as we use RDF to represent data, and SPARQL to interact with it internally. For more information see [Resource Description Framework](https://en.wikipedia.org/wiki/Resource_Description_Framework) and [SPARQL](https://en.wikipedia.org/wiki/SPARQL)

To create the mock data we use the following call.

In [6]:
from cognite.neat.graph import extractors

In [7]:
triples = extractors.MockGraphGenerator(power_rules).generate_triples(class_count)

We can now add these triples into the `neat` store

In [9]:
store.add_triples(triples)

We can now test the store by running the following query on it.

In [10]:
from cognite.neat.utils import remove_namespace

In [11]:
for res in list(store.graph.query("""
SELECT ?class (count(?s) as ?instances )
WHERE { ?s a ?class . }
GROUP BY ?class
ORDER BY DESC(?instances)""")):
    print(f"{remove_namespace(res[0]):25} {res[1]}" )

Terminal                  60
Substation                20
SubGeographicalRegion     10
GeographicalRegion        5


## Manual Extraction with Graph Capturing Sheet

Again, lets inspect the classes available in the *Transformation Rules*

In [12]:
power_rules.classes

,description,cdf_resource_type,deprecated,class_id,class_name,parent_asset
0,None,Asset,False,GeographicalRegion,GeographicalRegion,None
1,A subset of a geographical region of a power s...,Asset,False,SubGeographicalRegion,SubGeographicalRegion,GeographicalRegion
2,A substation is a part of an electrical genera...,Asset,False,Substation,Substation,SubGeographicalRegion
3,None,Asset,False,Terminal,Terminal,Substation


Let's now inspect properties related to one of the classes. Here we can see that class `Substation` contains two properties. First property in the list, `name`, contains value of type string, this type of property in semantic data modeling is known as data type properties, or in general graph theory this property is a node attribute, where node is equivalent to class instance. The second property, `subGeographicalRegon`, contains a link to `SubGeographicalRegion` instance. This type of property in the semantic data modeling is known as object properties, while in general graph theory this property represent an edge that connect nodes of two types.

In [13]:
columns = ["property_name", "property_type", "expected_value_type", "min_count", "max_count"]
power_rules.properties.groupby("class_id")["Substation"].to_pandas(include=columns)

,property_name,property_type,expected_value_type,min_count,max_count
0,name,DatatypeProperty,"{'prefix': 'xsd', 'suffix': 'string', 'name': ...",1,1
1,subGeographicalRegion,ObjectProperty,"{'prefix': 'power-grid', 'suffix': 'SubGeograp...",1,1


Now we can use the above data model to generate what we call `Graph Capturing Sheet` which is tailored Excel sheet containing:
- sheets for each of the defined classes
- columns corresponding to each property defined in data model

This sheet is generate using method `rules2graph_capturing_sheet` which is part of `extractors`. The method contains following arguments:

- `transformation_rules` : which is instance of transformation rules that contain definition of data model
- `file_path`: path where the graph capturing sheet should be stored
- `no_rows`: represent expected maximum number rows each sheet will have, thus corresponding to maximum of instance of any of define classes, by default set to 10000
- `auto_identifier_type` : type of auto identifier to be made for each class instance, by default set to `index` meaning `index-based` identifiers where index is row number
- `add_drop_down_list`: flag indicating whether to provide drop down selection of identifiers (i.e. links) for object type properties (i.e., edges)

We will use default values for arguments, meaning, automatic identifiers based on indexes, 10 000 rows, and drop down menus for object type properties:

In [13]:
from cognite.neat.graph import extractors

extractors.GraphCapturingSheet(power_rules, "power-grid-graph-capture.xlsx").create_template(overwrite=True)

In the animated gif below one can see how generated graph capturing sheet looks as well how process of capturing graph is conveyed.


<video src="../../videos/tutorial-2-graph-capturing-sheet.mp4" controls>
</video>


A row in a sheet represent an instance of a class. As one enters values for property in column `B`, the identifier is automatically added.
As we define instances, their identifier become in drop down menus for properties which are "edges" between "nodes". By connecting "nodes" we make a knowledge graph.

Let's now convert now filled graph capturing sheet into knowledge graph. First, we will create empty graph store object, then load raw sheet, and finally convert the raw sheet to graph using previously defined data model in transformation rules:

In [14]:
existing_sheet = extractors.GraphCapturingSheet(power_rules, "examples/power-grid-example.xlsx")

In [19]:
triples = existing_sheet.extract_triples_from_sheet()

In case you skipped the first part of this tutorial, we import the `NeatGraphStore` and initialize a new instance of it.

In [16]:
from cognite.neat.graph import NeatGraphStore

store = NeatGraphStore.from_rules(power_rules)

In [17]:
store.add_triples(triples)

To check graph content we can execute `SPARQL` to count all the class instances:
```
SELECT ?class (count(?s) as ?instances ) WHERE { ?s a ?class . } group by ?class order by DESC(?instances)
```

and later on when processing results we are purposely removing namespaces from the class names:

In [18]:
from cognite.neat.utils import remove_namespace

for res in list(store.graph.query("SELECT ?class (count(?s) as ?instances ) WHERE { ?s a ?class . } group by ?class order by DESC(?instances)")):
    print(f"{remove_namespace(res[0]):25} {res[1]}" )

GeographicalRegion        2
SubGeographicalRegion     2
Substation                2
Terminal                  2


As expected, we have two instances of each class that we captured through graph capturing sheet.